In [13]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
import os
from typing import Literal
from datetime import datetime
from pydantic import BaseModel

# --- Remote Ollama Server Configuration Notes (IMPORTANT!) ---
# For your remote Ollama server to be accessible via http://localhost:11434 on your client machine,
# you must have an SSH tunnel or similar port forwarding set up (e.g., `ssh -L 11434:localhost:11434 user@your_remote_server_ip`).
#
# On the remote server itself, before starting `ollama serve`, ensure these environment variables are set:
# export OLLAMA_HOST="0.0.0.0" # Allows Ollama to listen on all network interfaces
# export OLLAMA_ORIGINS="http://<your_client_ip_or_domain>,http://localhost:11434" # Allows CORS from your client
# Replace <your_client_ip_or_domain> with the actual IP or domain of the machine running this client code.
# If you are using an SSH tunnel, 'http://localhost:11434' might be sufficient for OLLAMA_ORIGINS on the server.

# Set the base URL for ChatOllama to your local forwarded port
# This assumes your SSH tunnel or port forwarding makes the remote Ollama accessible at this address.
OLLAMA_CLIENT_BASE_URL = "http://localhost:11434"

# --- Initialize ChatOllama instance ---
# This is your main LLM instance that will be used for both direct queries and tool calls.
llm_main = ChatOllama(
    model="llama3.1:latest", # Ensure this model is pulled on your remote Ollama server
    temperature=0.0,
    base_url=OLLAMA_CLIENT_BASE_URL, # Crucial: points to your accessible remote server (via tunnel) [1, 2]
    api_key="ollama" # Dummy value, as Ollama doesn't use API keys
)

# --- Define your tools ---
# Note: The llm_summarize_10words tool needs access to an LLM itself.
# We will pass the 'llm_main' instance to it.
# For tools that call the LLM internally, it's important that the internal LLM instance
# is correctly configured and available to the tool.
_llm_for_internal_tool_use = llm_main

@tool
def llm_summarize_10words(txt: str) -> str:
    """Summarize a text down to 10 words."""
    # This tool uses the main LLM instance to perform its summarization task.
    if _llm_for_internal_tool_use is None:
        raise RuntimeError("LLM for internal tool use not initialized.")
    # CORRECTED: Pass the HumanMessage with the text to summarize to the LLM
    res = _llm_for_internal_tool_use.invoke()
    return res.content

@tool
def check_10words(text: str) -> bool:
    """Check if a text contains exactly 10 words."""
    return len(text.split()) == 10

# --- Bind tools to the LLM instance ---
tools = [llm_summarize_10words, check_10words]
llm_with_tools = llm_main.bind_tools(tools) 

print(f"ChatOllama instance initialized with base_url: {llm_main.base_url}")
print(f"Tools bound: {[t.name for t in tools]}")

# --- Agentic Loop Demonstration ---

# Scenario 1: Query that should trigger a tool call (summarization)
print("\n--- Scenario 1: Invoking LLM with a tool-requiring query (Summarization) ---")
text_to_summarize = "LangChain is a framework designed to simplify the creation of applications using large language models. It provides tools, chains, and agents to build complex LLM workflows. It helps developers build context-aware, reasoning applications."
query_summarize = f"Please summarize the following text in exactly 10 words: '{text_to_summarize}'"

# Step 1: Invoke the LLM with the user's query
response_message_1 = llm_with_tools.invoke([HumanMessage(query_summarize)]) 
print(f"LLM's first response (AIMessage): {response_message_1}")
print(f"Content: '{response_message_1.content}'")
print(f"Tool Calls: {response_message_1.tool_calls}")

# Step 2: Check for tool calls and execute them
if response_message_1.tool_calls:
    print("\n--- Model decided to call a tool(s). Executing tool(s)... ---")
    messages_for_next_turn = [HumanMessage(query_summarize), response_message_1]

    for tool_call in response_message_1.tool_calls: # Corrected indentation [3, 5]
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        call_id = tool_call['id'] # Important for ToolMessage [3]

        print(f"  Executing Tool: '{tool_name}' with arguments: {tool_args}")

        # Map tool names to actual Python functions
        available_tools_map = {
            "llm_summarize_10words": llm_summarize_10words,
            "check_10words": check_10words
        }

        if tool_name in available_tools_map:
            selected_tool_func = available_tools_map[tool_name]
            try:
                # Execute the tool. For LangChain tools,.invoke() is the method.
                # Ensure arguments are passed correctly (e.g., as kwargs if tool expects them).
                tool_output = selected_tool_func.invoke(tool_args)
                print(f"  Tool Output: '{tool_output}'")

                # Append the ToolMessage to the conversation history [6, 7]
                messages_for_next_turn.append(
                    ToolMessage(content=str(tool_output), tool_call_id=call_id)
                )
            except Exception as e:
                print(f"  Error executing tool '{tool_name}': {e}")
                messages_for_next_turn.append(
                    ToolMessage(content=f"Error: {e}", tool_call_id=call_id)
                )
        else:
            print(f"  Error: Tool '{tool_name}' not found in available functions map.")
            messages_for_next_turn.append(
                ToolMessage(content=f"Error: Tool '{tool_name}' not found.", tool_call_id=call_id)
            )

    # Step 3: Re-invoke the LLM with the tool output for the final response
    print("\n--- Re-invoking LLM with tool output for final response ---")
    final_response_message = llm_with_tools.invoke(messages_for_next_turn)
    print(f"Final LLM Response (AIMessage): {final_response_message}")
    print(f"Final Content: '{final_response_message.content}'")
else:
    print("\nModel did not decide to call any tools for the first query. Content:")
    print(f"'{response_message_1.content}'")

# Scenario 2: Query that might trigger a different tool or no tool
print("\n--- Scenario 2: Invoking LLM with a query for check_10words ---")
text_to_check = "This is a test sentence with ten words exactly."
query_check_words = f"Check if the following text has exactly 10 words: '{text_to_check}'"

response_message_2 = llm_with_tools.invoke([HumanMessage(query_check_words)])
print(f"LLM's first response (AIMessage): {response_message_2}")
print(f"Content: '{response_message_2.content}'")
print(f"Tool Calls: {response_message_2.tool_calls}")

if response_message_2.tool_calls:
    print("\n--- Model decided to call a tool(s). Executing tool(s)... ---")
    messages_for_next_turn_2 = [HumanMessage(query_check_words), response_message_2]

    for tool_call in response_message_2.tool_calls: # Corrected indentation
        print("wtf")
        print(tool_call)
        print("ftw")
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        call_id = tool_call['id']

        print(f"  Executing Tool: '{tool_name}' with arguments: {tool_args}")
        available_tools_map = {
            "llm_summarize_10words": llm_summarize_10words,
            "check_10words": check_10words
        }

        if tool_name in available_tools_map:
            selected_tool_func = available_tools_map[tool_name]
            try:
                tool_output = selected_tool_func.invoke(tool_args)
                print(f"  Tool Output: '{tool_output}'")
                messages_for_next_turn_2.append(
                    ToolMessage(content=str(tool_output), tool_call_id=call_id)
                )
            except Exception as e:
                print(f"  Error executing tool '{tool_name}': {e}")
                messages_for_next_turn_2.append(
                    ToolMessage(content=f"Error: {e}", tool_call_id=call_id)
                )
        else:
            print(f"  Error: Tool '{tool_name}' not found in available functions map.")
            messages_for_next_turn_2.append(
                ToolMessage(content=f"Error: Tool '{tool_name}' not found.", tool_call_id=call_id)
            )

    print("\n--- Re-invoking LLM with tool output for final response ---")
    final_response_message_2 = llm_with_tools.invoke(messages_for_next_turn_2)
    print(f"Final LLM Response (AIMessage): {final_response_message_2}")
    print(f"Final Content: '{final_response_message_2.content}'")
else:
    print("\nModel did not decide to call any tools for the second query. Content:")
    print(f"'{response_message_2.content}'")

print("\n--- Example of invoking LLM with a general query (no tool expected) ---")
query_general = "What is an agent in the context of LLMs?"
response_general = llm_with_tools.invoke([HumanMessage(query_general)])
print(f"LLM Response (AIMessage): {response_general}")
print(f"Content: '{response_general.content}'")
print(f"Tool Calls: {response_general.tool_calls}")


ChatOllama instance initialized with base_url: http://localhost:11434
Tools bound: ['llm_summarize_10words', 'check_10words']

--- Scenario 1: Invoking LLM with a tool-requiring query (Summarization) ---
LLM's first response (AIMessage): content='' additional_kwargs={} response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-07-18T12:34:54.391660465Z', 'done': True, 'done_reason': 'stop', 'total_duration': 718137752, 'load_duration': 22615107, 'prompt_eval_count': 267, 'prompt_eval_duration': 3544864, 'eval_count': 64, 'eval_duration': 691356277, 'model_name': 'llama3.1:latest'} id='run--d41d7065-56f8-4bdb-934b-29b88f59ffc8-0' tool_calls=[{'name': 'llm_summarize_10words', 'args': {'txt': 'LangChain is a framework designed to simplify the creation of applications using large language models. It provides tools, chains, and agents to build complex LLM workflows. It helps developers build context-aware, reasoning applications.'}, 'id': '157eae0e-6f06-476d-bb5a-f29513271634', 'typ